In [1]:
import os

In [2]:
%pwd

'c:\\Users\\anasm\\OneDrive\\Desktop\\Image Processing\\Kidney disease detection\\research'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_include_top: bool
    params_classes: int
    param_weights: str

In [4]:
os.chdir('..')

In [5]:
from src.constants import *#config file path and parameters file path
from src.utils.common import read_yaml, create_directories

In [6]:
%pwd

'c:\\Users\\anasm\\OneDrive\\Desktop\\Image Processing\\Kidney disease detection'

In [7]:
class ConfigManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath= PARAM_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_directories(self.config.artifacts_root)

    def get_prepare_base_model_config(self) ->PrepareBaseModelConfig:
        config=self.config.prepare_base_model

        model_config=PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_classes=self.params.CLASSES,
            param_weights=self.params.WEIGHTS
        )

        return model_config


In [8]:
import os
import keras
import tensorflow as tf
from keras.layers import Input, Dense, Flatten
from keras import Model
from keras.applications.vgg16 import VGG16

In [9]:
class PrepareBaseModel:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config=config

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
    
    def get_base_model(self):
        conv_base=VGG16(weights='imagenet', include_top=False, input_shape=self.config.params_image_size)
        input_layer=Input(shape=self.config.params_image_size)
        x=conv_base(input_layer)
        self.model=Model(inputs=input_layer, outputs=x)
        self.model.trainable=False
        self.save_model(path=self.config.base_model_path, model=self.model)
        return self.model
    
    @staticmethod
    def prepare_full_model(model, classes):
        x=Flatten()(model.output)
        x=Dense(units=classes, activation='softmax')(x)
        full_model=Model(inputs=model.input, outputs=x)
        full_model.compile(loss='categorical_crossentropy', optimizer='adam')
        return full_model
    
    def update_base_model(self):
        self.full_model=self.prepare_full_model(
            model=self.model,
            classes=self.config.params_classes
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        return self.full_model

In [10]:
config_setter=ConfigManager()
model_config=config_setter.get_prepare_base_model_config()
print(model_config)

[2024-08-16 17:36:17,060: INFO: common: yaml file: C:\Users\anasm\OneDrive\Desktop\Image Processing\Kidney disease detection\config\config.yaml loaded successfully]
[2024-08-16 17:36:17,062: INFO: common: yaml file: C:\Users\anasm\OneDrive\Desktop\Image Processing\Kidney disease detection\params.yaml loaded successfully]
PrepareBaseModelConfig(root_dir=WindowsPath('artifacts/prepare_base_model'), base_model_path=WindowsPath('artifacts/prepare_base_model/base_model.h5'), updated_base_model_path=WindowsPath('artifacts/prepare_base_model/base_model_updated.h5'), params_image_size=BoxList([224, 224, 3]), params_include_top=False, params_classes=4, param_weights='imagenet')


In [11]:
model_generator=PrepareBaseModel(model_config)
base_model=model_generator.get_base_model()
base_model.summary()

[2024-08-16 17:36:17,431: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [12]:
full_cnn=model_generator.update_base_model()
full_cnn.summary()

[2024-08-16 17:36:17,566: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │       100,356 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,815,044 (56.51 MB)

 Trainable params: 100,356 (392.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)